In [26]:
import pycrfsuite
import numpy as np
from sklearn.metrics import classification_report
from itertools import chain
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import LabelBinarizer
import sklearn
import random

In [27]:
import numpy as np
import pandas as pd
import itertools

## load data accordingly Change in the Train-Validation Section Also
## Use : 
## sents0 for HI-ENG
## sents1 for BN_CR
## sents2 for TE_CR

In [99]:
def load_data(files):
    data, sent = [], []
    for file in files:
        with open(file, 'r') as rf:
            for line in rf:
                if line.strip() != '':
                    # Note: the shared corpus is already tokenized
                    sent.append(line.strip().split('\t'))
                else:
                    if len(sent) > 0:
                        data.append(sent)
                        sent = []
    return data

#sentmerge = load_data(['FB_HI_EN_CR.txt', 'TWT_HI_EN_CR.txt', 'WA_HI_EN_CR.txt'])
#sentmerge= load_data(['FB_BN_EN_CR.txt', 'TWT_BN_EN_CR.txt', 'WA_BN_EN_CR.txt'])
sentmerge= load_data(['FB_TE_EN_CR.txt', 'TWT_TE_EN_CR.txt', 'WA_TE_EN_CR.txt'])


#sentmerge = load_data(['FB_HI_EN_CR.txt', 'TWT_HI_EN_CR.txt', 'WA_HI_EN_CR.txt','FB_BN_EN_CR.txt', 'TWT_BN_EN_CR.txt', 'WA_BN_EN_CR.txt','FB_TE_EN_CR.txt', 'TWT_TE_EN_CR.txt', 'WA_TE_EN_CR.txt'])



In [100]:
# Train-Validation Set
random.seed(7)
random.shuffle(sentmerge)
train_sents = sentmerge[:int(0.8*len(sentmerge))]
valid_sents = sentmerge[int(0.8*len(sentmerge)):]
print("# Train sentences: %d" % (len(train_sents)))
print("# Validation sentences: %d" % (len(valid_sents)))

# Train sentences: 1583
# Validation sentences: 396


In [101]:
def word2features(sent, k):
    word = sent[k][0]
    features = [
        'token=%s' % (word)
    ]
    # extracting n-grams, for n=1 to 5
    for i in range(1,6):
        # if the value of n is greater than the word length, we exit the loop
        if i > len(word):
            break
        character_features = [word[j:j+i] for j in range(len(word)-i+1)]
        features.extend([
            # is count of individual n-grams important? is the order important?
            "char-%d-gram=%s" % (i, ' '.join(list(set(character_features))))
        ])
    if k == 0:
        # first word in the sentence
        features.append('BOS')
    else:
        features.extend([
            "-1:word=%s" % (sent[k-1][0])
        ])
    if i == len(sent):
        # last word in the sentence         
        features.append('EOS')
 
    return features
        
def sent2features(sent):
    # generating features for all the words/tokens in a sentence `sent`    
    return [word2features(sent, i) for i in range(len(sent))]

def sent2langs(sent):
    return [language_label for token, language_label, pos_tag in sent]

def sent2pos(sent):
    return [pos_tag for token, language_label, pos_tag in sent]

def sent2tokens(sent):
    return [token for token, language_label, pos_tag in sent]

In [102]:
X_train = [sent2features(sent) for sent in train_sents]
y_train = [sent2pos(sent) for sent in train_sents]

X_test = [sent2features(sent) for sent in valid_sents]
y_test = [sent2pos(sent) for sent in valid_sents]

In [103]:
trainer = pycrfsuite.Trainer(verbose=True)

for xseq, yseq in zip(X_train, y_train):
    trainer.append(xseq, yseq)

In [104]:
trainer.set_params({
    'c1': 0.1,   # coefficient for L1 penalty
    'c2': .01,# coefficient for L2 penalty
    'max_iterations': 200,  # stop earlier

    # include transitions that are possible, but not observed
    'feature.possible_transitions': True
})

In [105]:
trainer.train('icon2019_lid.crfsuite')

Feature generation
type: CRF1d
feature.minfreq: 0.000000
feature.possible_states: 0
feature.possible_transitions: 1
0....1....2....3....4....5....6....7....8....9....10
Number of features: 61029
Seconds required: 0.072

L-BFGS optimization
c1: 0.100000
c2: 0.010000
num_memories: 6
max_iterations: 200
epsilon: 0.000010
stop: 10
delta: 0.000010
linesearch: MoreThuente
linesearch.max_iterations: 20

***** Iteration #1 *****
Loss: 62730.834478
Feature norm: 1.000000
Error norm: 5430.024990
Active features: 60861
Line search trials: 1
Line search step: 0.000179
Seconds required for this iteration: 0.050

***** Iteration #2 *****
Loss: 50322.916525
Feature norm: 3.724598
Error norm: 3234.184675
Active features: 60823
Line search trials: 5
Line search step: 0.062500
Seconds required for this iteration: 0.126

***** Iteration #3 *****
Loss: 49978.810907
Feature norm: 7.896683
Error norm: 6245.834356
Active features: 59794
Line search trials: 1
Line search step: 1.000000
Seconds required for th

***** Iteration #44 *****
Loss: 7878.876538
Feature norm: 215.765413
Error norm: 27.369609
Active features: 52822
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.029

***** Iteration #45 *****
Loss: 7872.013410
Feature norm: 216.210933
Error norm: 19.612048
Active features: 52820
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.030

***** Iteration #46 *****
Loss: 7851.770812
Feature norm: 218.275556
Error norm: 77.260685
Active features: 52147
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.029

***** Iteration #47 *****
Loss: 7851.434826
Feature norm: 218.878589
Error norm: 194.454656
Active features: 52007
Line search trials: 2
Line search step: 0.500000
Seconds required for this iteration: 0.054

***** Iteration #48 *****
Loss: 7839.347522
Feature norm: 219.084167
Error norm: 47.266903
Active features: 52141
Line search trials: 1
Line search step: 1.000000
Seconds 

***** Iteration #89 *****
Loss: 7697.746827
Feature norm: 233.159158
Error norm: 8.377435
Active features: 47335
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.029

***** Iteration #90 *****
Loss: 7697.126630
Feature norm: 233.161726
Error norm: 15.869681
Active features: 47277
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.029

***** Iteration #91 *****
Loss: 7696.258107
Feature norm: 233.190612
Error norm: 16.692320
Active features: 47232
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.030

***** Iteration #92 *****
Loss: 7695.544099
Feature norm: 233.190273
Error norm: 18.921287
Active features: 47211
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.030

***** Iteration #93 *****
Loss: 7694.941805
Feature norm: 233.211716
Error norm: 20.814229
Active features: 47162
Line search trials: 1
Line search step: 1.000000
Seconds re

***** Iteration #129 *****
Loss: 7682.928655
Feature norm: 234.579336
Error norm: 5.043459
Active features: 46302
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.030

***** Iteration #130 *****
Loss: 7682.900293
Feature norm: 234.629701
Error norm: 13.226632
Active features: 46288
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.028

***** Iteration #131 *****
Loss: 7682.580789
Feature norm: 234.666096
Error norm: 8.867500
Active features: 46274
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.030

***** Iteration #132 *****
Loss: 7682.473106
Feature norm: 234.691857
Error norm: 9.574981
Active features: 46262
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.030

***** Iteration #133 *****
Loss: 7682.284331
Feature norm: 234.727668
Error norm: 4.243282
Active features: 46256
Line search trials: 1
Line search step: 1.000000
Seconds 

***** Iteration #175 *****
Loss: 7677.926212
Feature norm: 235.661919
Error norm: 8.312716
Active features: 45838
Line search trials: 2
Line search step: 0.500000
Seconds required for this iteration: 0.054

***** Iteration #176 *****
Loss: 7677.900487
Feature norm: 235.667399
Error norm: 6.001785
Active features: 45835
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.030

***** Iteration #177 *****
Loss: 7677.793075
Feature norm: 235.692885
Error norm: 3.338563
Active features: 45833
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.029

***** Iteration #178 *****
Loss: 7677.746729
Feature norm: 235.701014
Error norm: 7.476507
Active features: 45835
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.029

***** Iteration #179 *****
Loss: 7677.700984
Feature norm: 235.711280
Error norm: 9.342669
Active features: 45829
Line search trials: 1
Line search step: 1.000000
Seconds r

In [106]:
tagger = pycrfsuite.Tagger()
tagger.open('icon2016_lid.crfsuite')

In [107]:
y_true = []
y_pred = []
for _ in valid_sents:
    new_sent = _
    [y_pred.append(_0) for _0 in tagger.tag(sent2features(new_sent))]
    [y_true.append(_1) for _1 in sent2pos(new_sent)]

In [108]:
print(classification_report(y_true,y_pred,digits=2))

              precision    recall  f1-score   support

           #       1.00      0.04      0.09        45
           $       0.89      0.71      0.79        66
           @       0.91      0.20      0.33       253
          CC       0.74      0.73      0.73        70
          DT       0.84      0.71      0.77       107
           E       0.93      0.79      0.85        52
         G_J       0.89      0.38      0.54       317
         G_N       0.55      0.97      0.70      2140
       G_PRP       0.89      0.47      0.61       243
       G_PRT       0.88      0.36      0.51       140
         G_R       0.85      0.42      0.56       133
       G_SYM       1.00      0.11      0.20         9
         G_V       0.82      0.41      0.55       601
         G_X       0.91      0.61      0.73      1254
         PSP       0.88      0.53      0.66       279
           U       0.89      0.45      0.60        55
        null       0.81      0.21      0.33        63
           ~       1.00    

# Test Set 

In [109]:
sents0 = load_data(['EN_HI_test.txt'])
sents1 = load_data(['EN_BN_test.txt'])
sents2 = load_data(['EN_TE_test.txt'])

test_sents = sents0
test_sents1 = sents1
test_sents2 = sents2


## EN_HI_test

In [42]:
#sents0

In [54]:
y_true = []
y_pred = []
for _ in test_sents:
    new_sent = _
    [y_pred.append(_0) for _0 in tagger.tag(sent2features(new_sent))]
    #[y_true.append(_1) for _1 in sent2pos(new_sent)]

In [55]:
y_pred

['G_N',
 'G_N',
 'G_X',
 'DT',
 'G_N',
 'PSP',
 'DT',
 'G_N',
 'G_N',
 'G_X',
 'G_N',
 'G_N',
 'G_V',
 'G_N',
 'G_X',
 'G_V',
 'G_N',
 'G_N',
 'G_N',
 'G_X',
 'G_N',
 'PSP',
 'G_V',
 'PSP',
 'G_N',
 'G_N',
 'G_N',
 'G_N',
 'G_N',
 'G_N',
 'G_N',
 'G_X',
 'G_N',
 'G_N',
 'G_N',
 'G_N',
 'G_N',
 'G_N',
 'G_N',
 'G_N',
 'G_N',
 'G_N',
 'G_N',
 'G_N',
 'G_N',
 'G_N',
 'G_N',
 'G_X',
 'G_N',
 'G_N',
 'G_N',
 'G_N',
 'G_X',
 'G_N',
 'G_N',
 'G_N',
 'G_N',
 'G_N',
 'G_J',
 'G_V',
 'PSP',
 'G_N',
 'G_N',
 'G_N',
 'E',
 'E',
 'E',
 'E',
 'G_N',
 'G_N',
 'G_N',
 'G_X',
 'G_N',
 'G_X',
 'G_N',
 'G_N',
 'G_V',
 'G_V',
 'G_N',
 'G_N',
 'G_N',
 'G_V',
 'CC',
 'G_N',
 'G_X',
 'G_X',
 'G_N',
 'G_N',
 'G_V',
 'G_V',
 'G_V',
 'G_X',
 'G_X',
 'G_N',
 'G_V',
 'G_N',
 'G_N',
 'G_V',
 'G_X',
 'G_V',
 'G_PRP',
 'G_V',
 'G_N',
 'G_N',
 'G_N',
 'G_N',
 'G_X',
 '#',
 '#',
 'G_N',
 'G_N',
 'G_N',
 'G_N',
 'G_N',
 'G_N',
 'G_N',
 'PSP',
 'G_J',
 'G_N',
 'G_V',
 'G_N',
 'G_N',
 'PSP',
 'G_N',
 'G_N',
 'G_N',
 'G_N

In [47]:
print(len(sents0))
sents0

440


[[['Shoib', 'ne'],
  ['Akhtar', 'ne'],
  ['..', 'univ'],
  ['A', 'en'],
  ['man', 'en'],
  ['with', 'en'],
  ['a', 'en'],
  ['Midas', 'en'],
  ['touch', 'en'],
  ['...', 'univ'],
  ['Played', 'en'],
  ['cricket', 'en'],
  ['got', 'en'],
  ['famous', 'en'],
  [',', 'univ'],
  ['doing', 'en'],
  ['commentary', 'en'],
  ['par', 'en'],
  ['excellence', 'en'],
  ['.', 'univ'],
  ['Something', 'en'],
  ['to', 'en'],
  ['cheer', 'en'],
  ['about', 'en'],
  ['😂😂😋', 'univ'],
  ['#INDvSA', 'univ']],
 [['#IndvsSA', 'univ'],
  ['Shoib', 'ne'],
  ['Akhtar', 'ne'],
  ['YEH', 'hi'],
  ['DEKHIYE', 'hi'],
  [',', 'univ'],
  ['YEH', 'hi'],
  ['DEKHIYE', 'hi'],
  ['AUR', 'hi'],
  ['YEH', 'hi'],
  ['BHI', 'hi'],
  ['DEKHIYE', 'hi'],
  ['#WorldCup2015', 'univ'],
  ['hindi', 'hi'],
  ['commentary', 'en'],
  ['AARE', 'hi'],
  ['BHAI', 'hi'],
  ['HUM', 'hi'],
  ['DEKH', 'hi'],
  ['RAHEIN', 'hi'],
  ['HAIN', 'hi'],
  ['...', 'univ']],
 [['Indian', 'en'],
  ['Bowling', 'en'],
  ['ka', 'hi'],
  ['Dewaana', 'hi']

In [61]:
flat_list = []
for sublist in sents0:
    for item in sublist:
        flat_list.append(item)
flat_list

[['Shoib', 'ne'],
 ['Akhtar', 'ne'],
 ['..', 'univ'],
 ['A', 'en'],
 ['man', 'en'],
 ['with', 'en'],
 ['a', 'en'],
 ['Midas', 'en'],
 ['touch', 'en'],
 ['...', 'univ'],
 ['Played', 'en'],
 ['cricket', 'en'],
 ['got', 'en'],
 ['famous', 'en'],
 [',', 'univ'],
 ['doing', 'en'],
 ['commentary', 'en'],
 ['par', 'en'],
 ['excellence', 'en'],
 ['.', 'univ'],
 ['Something', 'en'],
 ['to', 'en'],
 ['cheer', 'en'],
 ['about', 'en'],
 ['😂😂😋', 'univ'],
 ['#INDvSA', 'univ'],
 ['#IndvsSA', 'univ'],
 ['Shoib', 'ne'],
 ['Akhtar', 'ne'],
 ['YEH', 'hi'],
 ['DEKHIYE', 'hi'],
 [',', 'univ'],
 ['YEH', 'hi'],
 ['DEKHIYE', 'hi'],
 ['AUR', 'hi'],
 ['YEH', 'hi'],
 ['BHI', 'hi'],
 ['DEKHIYE', 'hi'],
 ['#WorldCup2015', 'univ'],
 ['hindi', 'hi'],
 ['commentary', 'en'],
 ['AARE', 'hi'],
 ['BHAI', 'hi'],
 ['HUM', 'hi'],
 ['DEKH', 'hi'],
 ['RAHEIN', 'hi'],
 ['HAIN', 'hi'],
 ['...', 'univ'],
 ['Indian', 'en'],
 ['Bowling', 'en'],
 ['ka', 'hi'],
 ['Dewaana', 'hi'],
 ['=', 'univ'],
 ['Shoib', 'ne'],
 ['Akhtar', 'ne'],

In [78]:
#for i in y_pred:
#    final = [x + [i] for x in flat_list]
#final
newpred = pd.DataFrame(y_pred)
newflatlist = pd.DataFrame(flat_list)
final = newflatlist
final["2"] = newpred
final


,0,1,2
0,Shoib,ne,G_N
1,Akhtar,ne,G_N
2,..,univ,G_X
3,A,en,DT
4,man,en,G_N
5,with,en,PSP
6,a,en,DT
7,Midas,en,G_N
8,touch,en,G_N
9,...,univ,G_X


In [79]:
fin = pd.DataFrame(final)
fin

,0,1,2
0,Shoib,ne,G_N
1,Akhtar,ne,G_N
2,..,univ,G_X
3,A,en,DT
4,man,en,G_N
5,with,en,PSP
6,a,en,DT
7,Midas,en,G_N
8,touch,en,G_N
9,...,univ,G_X


In [80]:
np.savetxt('EN_HI_test_AI0002.txt', fin.values, fmt='%s', delimiter="\t")  

## EN_BN_test

In [92]:
test_sents1

[[['Ami', 'bn'],
  ['Besh', 'bn'],
  ['Kichu', 'bn'],
  ['Comments', 'en'],
  ['dakha', 'bn'],
  ['obak', 'bn'],
  ['holam', 'bn'],
  ['....', 'univ'],
  ['Keo', 'bn'],
  ['Inspired', 'en'],
  ['bapartakeo', 'bn'],
  ['ektu', 'bn'],
  ['onno', 'bn'],
  ['vabha', 'bn'],
  ['tule', 'bn'],
  ['dhorte', 'bn'],
  ['chaicha', 'bn'],
  ['.', 'univ'],
  ['Amr', 'bn'],
  ['mone', 'bn'],
  ['hoy', 'bn'],
  ['kichu', 'bn'],
  ['valo', 'bn'],
  ['kaj', 'bn'],
  ['korar', 'bn'],
  ['jonno', 'bn'],
  ['jodi', 'bn'],
  ['onno', 'bn'],
  ['karo', 'bn'],
  ['thake', 'bn'],
  ['Inspired', 'en'],
  ['hote', 'bn'],
  ['hoy', 'bn'],
  ['ate', 'bn'],
  ['kono', 'bn'],
  ['khoto', 'bn'],
  ['nei', 'bn'],
  ['....', 'univ'],
  ['Borog', 'bn'],
  ['onek', 'bn'],
  ['lav', 'bn'],
  ['acha', 'bn'],
  ['...', 'univ'],
  ['Ektu', 'bn'],
  ['vabha', 'bn'],
  ['dekhun', 'bn'],
  ['amr', 'bn'],
  ['kotha', 'bn'],
  ['ta', 'bn'],
  ['.', 'univ'],
  ['R', 'bn'],
  ['desh', 'bn'],
  ['er', 'bn'],
  ['kotha', 'bn'],
  ['

In [93]:
y_true = []
y_pred = []
for _ in test_sents1:
    new_sent = _
    [y_pred.append(_0) for _0 in tagger.tag(sent2features(new_sent))]
    #[y_true.append(_1) for _1 in sent2pos(new_sent)]

In [94]:
y_pred

['G_PRP',
 'G_N',
 'G_N',
 'G_N',
 'G_N',
 'G_N',
 'G_N',
 'G_X',
 'G_N',
 'G_N',
 'G_N',
 'G_N',
 'G_N',
 'G_X',
 'G_N',
 'G_N',
 'G_N',
 'G_X',
 'G_PRP',
 'G_N',
 'G_V',
 'G_N',
 'G_V',
 'G_V',
 'G_V',
 'G_PRP',
 'G_V',
 'G_PRP',
 'G_V',
 'G_N',
 'G_N',
 'G_V',
 'G_V',
 'G_N',
 'G_N',
 'G_N',
 'G_V',
 'G_X',
 'G_N',
 'G_N',
 'G_N',
 'G_J',
 'G_X',
 'G_N',
 'G_N',
 'G_N',
 'G_PRP',
 'G_N',
 'G_PRT',
 'G_X',
 'CC',
 'G_N',
 'G_PRP',
 'G_N',
 'G_N',
 'G_N',
 'G_N',
 'G_PRP',
 'G_N',
 'G_V',
 'G_N',
 'G_N',
 'PSP',
 'G_N',
 'G_N',
 'G_N',
 'G_N',
 'G_N',
 'G_N',
 'PSP',
 'G_N',
 'G_X',
 'G_PRP',
 'G_N',
 'G_N',
 'PSP',
 'G_N',
 'G_N',
 'G_N',
 'G_N',
 'G_N',
 'G_N',
 'G_PRP',
 'G_SYM',
 'G_N',
 'G_N',
 'G_N',
 'G_PRT',
 'G_X',
 'G_N',
 'G_N',
 'G_N',
 'G_N',
 'G_N',
 'G_V',
 'G_N',
 'G_N',
 'G_N',
 'G_N',
 'PSP',
 'G_N',
 'G_N',
 'G_N',
 'G_N',
 'DT',
 'G_N',
 'G_N',
 'G_PRP',
 'G_N',
 'G_N',
 'G_N',
 'G_N',
 'G_N',
 'G_V',
 'G_V',
 'G_N',
 'G_N',
 'G_N',
 'G_V',
 'G_X',
 'G_V',
 'DT',
 

In [95]:
flat_list = []
for sublist in sents1:
    for item in sublist:
        flat_list.append(item)

In [96]:
newpred = pd.DataFrame(y_pred)
newflatlist = pd.DataFrame(flat_list)
final = newflatlist
final["2"] = newpred
final

,0,1,2
0,Ami,bn,G_PRP
1,Besh,bn,G_N
2,Kichu,bn,G_N
3,Comments,en,G_N
4,dakha,bn,G_N
5,obak,bn,G_N
6,holam,bn,G_N
7,....,univ,G_X
8,Keo,bn,G_N
9,Inspired,en,G_N


In [23]:
#for item in y_pred:
 #   final = [x + [item] for x in flat_list]
#final

[['Ami', 'bn', 'G_PRP'],
 ['Besh', 'bn', 'G_PRP'],
 ['Kichu', 'bn', 'G_PRP'],
 ['Comments', 'en', 'G_PRP'],
 ['dakha', 'bn', 'G_PRP'],
 ['obak', 'bn', 'G_PRP'],
 ['holam', 'bn', 'G_PRP'],
 ['....', 'univ', 'G_PRP'],
 ['Keo', 'bn', 'G_PRP'],
 ['Inspired', 'en', 'G_PRP'],
 ['bapartakeo', 'bn', 'G_PRP'],
 ['ektu', 'bn', 'G_PRP'],
 ['onno', 'bn', 'G_PRP'],
 ['vabha', 'bn', 'G_PRP'],
 ['tule', 'bn', 'G_PRP'],
 ['dhorte', 'bn', 'G_PRP'],
 ['chaicha', 'bn', 'G_PRP'],
 ['.', 'univ', 'G_PRP'],
 ['Amr', 'bn', 'G_PRP'],
 ['mone', 'bn', 'G_PRP'],
 ['hoy', 'bn', 'G_PRP'],
 ['kichu', 'bn', 'G_PRP'],
 ['valo', 'bn', 'G_PRP'],
 ['kaj', 'bn', 'G_PRP'],
 ['korar', 'bn', 'G_PRP'],
 ['jonno', 'bn', 'G_PRP'],
 ['jodi', 'bn', 'G_PRP'],
 ['onno', 'bn', 'G_PRP'],
 ['karo', 'bn', 'G_PRP'],
 ['thake', 'bn', 'G_PRP'],
 ['Inspired', 'en', 'G_PRP'],
 ['hote', 'bn', 'G_PRP'],
 ['hoy', 'bn', 'G_PRP'],
 ['ate', 'bn', 'G_PRP'],
 ['kono', 'bn', 'G_PRP'],
 ['khoto', 'bn', 'G_PRP'],
 ['nei', 'bn', 'G_PRP'],
 ['....', 'un

In [97]:
fin = pd.DataFrame(final)
np.savetxt('EN_BN_test_AI0002.txt', fin.values, fmt='%s', delimiter="\t")  

# EN_TE_test

In [110]:
y_true = []
y_pred = []
for _ in test_sents2:
    new_sent = _
    [y_pred.append(_0) for _0 in tagger.tag(sent2features(new_sent))]
    #[y_true.append(_1) for _1 in sent2pos(new_sent)]

In [111]:
flat_list = []
for sublist in sents2:
    for item in sublist:
        flat_list.append(item)

In [112]:
newpred = pd.DataFrame(y_pred)
newflatlist = pd.DataFrame(flat_list)
final = newflatlist
final["2"] = newpred
final

,0,1,2
0,Mahesh,univ,G_N
1,babu,univ,G_N
2,n,univ,CC
3,Rajamouli,univ,G_N
4,movie,en,G_N
5,confirm,en,G_N
6,ah,univ,G_PRT
7,.?,univ,G_X
8,details,univ,G_N
9,??,univ,G_X


In [113]:
#for item in y_pred:
#    final = [x + [item] for x in flat_list]
#final

In [114]:
fin = pd.DataFrame(final)
np.savetxt('EN_TE_test_AI0002.txt', fin.values, fmt='%s', delimiter="\t")  